This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [10]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'hsn53PAYpmVYWmq5PT8D'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [105]:
# First, import the relevant modules
import requests
import numpy as np

In [72]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-02&end_date=2017-01-02&api_key=hsn53PAYpmVYWmq5PT8D"
one_day = requests.get(url).json()

In [73]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
one_day

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-05-15) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-05-29T22:16:21.973Z',
  'newest_available_date': '2020-05-29',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-02',
  'end_date': '2017-01-02',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## 1. Collect AFX_X data for 2017

In [71]:
url = f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=hsn53PAYpmVYWmq5PT8D"
result = requests.get(url).json()

## 2. Convert the returned JSON object into a Python dictionary.

In [76]:
result = dict(result)

## 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [87]:
# Get the data from the dictionary
data = result["dataset"]["data"]

# Print the column names of the dataset
print(result["dataset"]["column_names"])


['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [116]:
# The opening price is the 1st index in each sublist, removing None entries
opens = [sublist[1] for sublist in data if sublist[1] is not None]

In [117]:
print(f"Highest opening price: {max(opens)}")
print(f"Lowest opening price: {min(opens)}")

Highest opening price: 53.11
Lowest opening price: 34.0


## 4. What was the largest change in any one day (based on High and Low price)?

In [133]:
# Highest and lowest are in the 2nd and 3rd indeces of each sublist. Don't return if high or low is None.
highs, lows = zip(*[(sublist[2], sublist[3]) for sublist in data if (sublist[2] is not None) and (sublist[3] is not None)])

# Convert the returned tuples into arrays for fast computation
highs = np.array(highs); lows = np.array(lows)

# Convert the change for each day
diff = max(highs-lows)
print(f"Largest daily change: ${round(diff, 2)}")

Largest daily change: $2.81


## 5. What was the largest change between any two days (based on Closing Price)?

In [146]:
# going to assume this means any two consecutive days
closes = [sublist[4] for sublist in data if sublist[4] is not None]

# let's get the absolute change, since we only care about the change, not necessarily if it's +/-
max_diff = 0
for idx, close in enumerate(closes):
    
    # skip the first close
    if idx == 0:
        prev_close = close
        continue
    
    curr_diff = abs(close - prev_close)
    
    if curr_diff > max_diff:
        max_diff = curr_diff
    
    # Store the current close as previous close for next computation
    prev_close = close
    
print(f"Largest consecutive difference: ${round(max_diff, 2)}")

Largest consecutive difference: $2.56


## 6. What was the average daily trading volume during this year?

In [154]:
traded_volume = [sublist[6] for sublist in data if sublist[6] is not None]
avg_volume = sum(traded_volume)/len(traded_volume)
print(f"Average daily trading volume: {round(avg_volume):,}")

Average daily trading volume: 89,124
